In [1]:
# !pip install scrapy
# !pip install seleniumnn 
# !pip install beautifulsoup4 

In [2]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import re
import time
import csv
from itertools import islice
import pandas as pd
import os

In [3]:
driver = webdriver.Chrome('./chromedriver')

In [4]:
driver.get(f'https://www.6pm.com/marty/c/homepage-new')

In [5]:
def get_cookie_jar(driver):
    cookies = driver.get_cookies()
    cookie_jar = {x['name']:x['value'] for x in cookies}
    
    return cookie_jar

cookie_jar = get_cookie_jar(driver)

In [6]:
# Pass cookies back to request page information
page = requests.get('https://www.6pm.com/marty/c/homepage-new', 
                    cookies = cookie_jar)

In [7]:
soup = BeautifulSoup(page.text, 'html.parser')

In [8]:
boot_page = soup.findAll('a',{'class':'gae-click*Main-Nav*Shoes*Women-s-Boots'})

In [9]:
boot_page

[<a class="gae-click*Main-Nav*Shoes*Women-s-Boots" href="/women-boots/CK_XARCz1wHAAQHiAgMYAQI.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/">Boots</a>]

In [10]:
boot_page_str = str(boot_page)
start = boot_page_str.find('href') + 6
end = boot_page_str.find('Boots</a>') - 2
boot_page_str[start:end]

'/women-boots/CK_XARCz1wHAAQHiAgMYAQI.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/'

In [11]:
URL = "http://6pm.com" + boot_page_str[start:end]
page1 = requests.get(URL, cookies = cookie_jar)
page1

<Response [200]>

In [12]:
driver.get(URL)

In [13]:
driver.current_url

'https://www.6pm.com/women-boots/CK_XARCz1wHAAQHiAgMBAhg.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/'

In [14]:
# Need to add the size "5.5" click later, now we just copy and paste it to get around it

In [15]:
URL2 = "https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/"

In [16]:
URL2

'https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/'

In [17]:
def extract_links_from_soup(current_soup):
    links = []
    for link in current_soup.find_all('a', {'itemprop':'url'}):
        links.append(link.get('href'))  
    return links

In [18]:
def fill_images_to_dataframe(links, pd_images):
    for link in links:
        URL_current = "http://www.6pm.com" + link
#        print("current_search_page: ", URL_current)
        page_current = requests.get(URL_current, cookies = cookie_jar)
        soup_current = BeautifulSoup(page_current.text, 'html.parser')
#        print(soup_current.text)
        try: 
            image_link = soup_current.find('img',{'alt':'BOTT'})['src']
        except TypeError:
            print("This item doesn't have a sole picture!")
        else:
            image_name = URL_current.split("/")[4] + ".jpg"
#            print(image_name)
#            print(image_link)
            pd_shoe_current = pd.Series([image_link, image_name, link], index=['image_link', 'jpeg', 'shoes_link'])
            pd_images = pd_images.append(pd_shoe_current, ignore_index=True)
    return pd_images

In [19]:
#soup_current.find('img',{'alt':'BOTT'})['src']

In [20]:
#fill_images_to_dataframe(links, pd_images)

In [21]:
def save_jpegs(pd_images):
    for index, row in pd_images.iterrows():
        with open(os.path.join('./Shoe_sole_images_2/',row['jpeg']), "wb") as file1:
            image = requests.get(row['image_link'])
            file1.write(image.content)
            file1.close()

In [22]:
 #   current_URL = "https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=27"

In [34]:
current_URL = "https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=153"
signal = 0
pd_images = pd.DataFrame(columns=['image_link', 'jpeg', 'shoes_link'])
while(signal == 0):
    current_page = requests.get(current_URL, cookies = cookie_jar)
    print("current URL: ", current_URL)
    current_soup = BeautifulSoup(current_page.text, 'html.parser')
    links = extract_links_from_soup(current_soup)
    pd_images = fill_images_to_dataframe(links, pd_images)
    save_jpegs(pd_images)
    time.sleep(3)
    try: 
        current_URL = "https://www.6pm.com" + current_soup.findAll('a', {'rel': 'next'})[0]['href']
    except:
        print("Key Error: End of Search!")
        signal = 1
        
    

current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=153
current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=154
This item doesn't have a sole picture!
current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=155
current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=156
current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=157
current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/&p=158
current URL:  https://www.6pm.com/women-shoes/CK_XAXoC1ASCAQKfGMABAeICAwEYDw.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/

ConnectionError: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/81JaLPuS0+L._SR106,78_.jpg (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1160cb080>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
## pd_images.shape

In [ ]:
current_URL

In [ ]:
pd_images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
im.show()


In [ ]:
# numpy arrays

In [ ]:
plt.imshow()

In [ ]:
# result = soup_current.find_all('img',{'alt':'BOTT'})
# for x in result:
#    print(x['src'])

# soup_current.find('img',{'alt':'BOTT'})['src']